## 13.5 향상된 비교 연산자

#### 예제 10.16

In [1]:
from array import array
import reprlib
import math
import functools
import operator
import itertools

class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    


    def __len__(self):
        return len(self._components)

    shortcut_names = 'xyzt'

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))
    
    def __getattr__(self, name):
        cls = type(self)  # Vector 클래스 가져오기
        if len(name) == 1:  # name이 한 글자이면 shortcut_names 들 중 하나 일 수 있음
            pos = cls.shortcut_names.find(name)  # 한 글자 name의 위치 찾음
            if 0 <= pos < len(self._components):  # <4>
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'  # <5>
        raise AttributeError(msg.format(cls, name))

    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:  # 단일 문자 속성명에 대해 특별한 처리를 함
            if name in cls.shortcut_names:  # name이 x, y, z, t 중 하나이면 구체적인 에러메세지
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():  # <3> # 그 외 소문자이면
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else: # 그렇지 않으면
                error = ''  # <4>
            if error:  # <5># error 안에 어떤 문자가 들어있으면 
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)  # 에러가 발생하지 않을 때는 표준 동작을 위해 슈퍼클래스의 __setattr__() 호출

        
    def __eq__(self, other):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))
    
    def __hash__(self):
        hashes = map(hash, self._components)
        return functools.reduce(operator.xor, hashes, 0)
    
    
    def angle(self, n):  # 초구면좌표에 대한 공식을 이용해서 특정 좌표에 대한 각 좌표를 계산
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):  # 요청에 다라 각좌표를 모두 계산하는 제너레이터 표현식 생성
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        print(self)
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())  # itertools.chain 함수릴 이용해 크기와 각좌표를 차례대로 반복하는 제너레이터 표현식 생성
            outer_fmt = '<{}>'  # 구면 좌표 출력 설정
        else:
            coords = self
            outer_fmt = '({})'  # 직교좌표 출력 설정
        components = (format(c, fmt_spec) for c in coords)  # 좌표의 각 항목을 요청에 따라 포맷하는 제너레이터 생성
        return outer_fmt.format(', '.join(components))  # <8> 포맷된 요소들을 콤마로 분리해서 꺽쇠 괄호나 괄호 안에 넣음

#### 예제 13-12 Vector를 Vector, Vector2d, 튜플과 비교하기


In [2]:
va = Vector([1.0, 2.0, 3.0])
vb = Vector(range(1, 4))
va == vb # 동일한 숫자 요소를 가진 두 Vector를 동일하다고 판단

True

In [3]:
vc = Vector([1, 2])

from vector2d_v3 import Vector2d

v2d = Vector2d(1, 2)
print(v2d == vc) # 요소의 값이 같다면 Vector2d 와 Vector2도 동일하다고 판단
# vc == v2d는 실행 안됨

t3 = (1, 2, 3)
print(va == t3) # Vector가 동일한 값의 숫자 항목을 가진 튜플이나 여타 반복형과도 동일하다고 판단

True
True


<hr>

- 경우에 따라 위 예제의 마지막 결과는 바람직하지 않을 수 있음
    - 이에 대한 엄격한 규칙은 없으면, 애플리케이션에 따라 다름
    - "모호함에 직면할 때는 추측하려는 유혹을 거부하라"
- 피연산자를 평가할 때 지나친 자유분방함은 예기치 못한 결과를 낳을 수 있음

#### 예제 13-13 vector_v8.py : Vector 클래스의 \__equl\__() 매서드 개선

In [4]:
from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))
    
    ####################################### 수정합니다 #######################################    
    
    def __eq__(self, other):
        if isinstance(other, Vector): # other 피 연산자가 Vector나 Vector 서브클래스의 객체면 기존과 동일하게 비교
            return (len(self)==len(other) and all(a==b for a, b in zip(self, other)))
        else:
            return NotImplemented
        
    ###########################################################################################

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __neg__(self):
        return Vector(-x for x in self)

    def __pos__(self):
        return Vector(self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented

    def __radd__(self, other):
        return self + other

    def __mul__(self, scalar):
        if isinstance(scalar, numbers.Real):
            return Vector(n * scalar for n in self)
        else:
            return NotImplemented

    def __rmul__(self, scalar):
        return self * scalar

#### 예제 13-14 

In [5]:
va = Vector([1.0, 2.0, 3.0])
vb = Vector(range(1, 4))
va == vb # 동일한 숫자 요소를 가진 두 Vector를 동일하다고 판단

True

In [6]:
vc = Vector([1, 2])

v2d = Vector2d(1, 2)
print(vc==v2d) 

t3 = (1, 2, 3)
print(va == t3) # Vector가 동일한 값의 숫자 항목을 가진 튜플이나 여타 반복형과도 동일하다고 판단

True
False


<hr>

**Vector와 Vector2d 객체에서 비교**
1. vc == v2d를 평가하기 위해 파이썬은 Vector.\__eq\__(vc, v2d) 호출
2. Vector.\__eq\__(vc, v2d) 는 v2d가 Vector 객체가 아님을 확인하고 NotImplemented 반환
3. NotImplemented가 반환되었으므로 파이썬은 Vector2d.\__eq\__(v2d, vc) 실행
4. Vector2d.\__eq\__(v2d, vc)는 피연산자 두 개를 모듀 튜플로 변환하여 비교

**Vector와 tuple 비교**
1. vc == t3 를 평가하기 위해 파이썬은 Vector.\__eq\__(va, t3) 호출
2. Vector.\__eq\__(va, t3) 는 t3가 Vector 객체가 아님을 확인하고 NotImplemented 반환
3. NotImplemented를 받은 파이썬 인터프리터는 tuple.\__eq\__(t3, va) 시도
4. tuple.\__eq\__(t3, va) 은 Vector 형에 대해 알지 못하므로 NotImplemented 반환
5. == 연산자의 경우 특별히 역순 메서드가 NotImplemented를 반환하면 파이썬 인터프리터는 최후의 수단으로 두 아이디의 객체 ID 비교

!= 연산자
- \__ne\__() 메서드가 우리 목적에 맞게 처리해주므로 우리가 직접 구현할 필요는 없음
- \__eq\__() 메서드가 구현되어 있고 NotImplemented를 반환하지 않으면, \__ne\__()는 \__eq\__()가 반환한 반댓값을 반환

In [7]:
print(va != vb)
print(vc != v2d)
print(va != (1, 2, 3))

False
False
True


In [8]:
def __ne__(self, other):
    eq_result = self == otehr
    if eq_result is NotImplemented:
        return NotImplemented
    else:
        return not eq_result

## 13.6 복합 할당 연산자

#### 예제 13-15 복합 할당이 불변 타깃을 처리할 때는 객체를 새로 생성하고 다시 바인딩 함

In [9]:
v1 = Vector([1, 2, 3])
v1_alias = v1 # 별명을 생성해서 Vector([1, 2, 3]) 객체를 나중에 다시 조회할 수 있도록 
print(id(v1), id(v1_alias)) # 원래 Vector 객체의 ID는 v1에 바인딩

2315758265288 2315758265288


In [10]:
v1 += Vector([4, 5, 6]) # 덧셈 할당자 실행
v1 

Vector([5.0, 7.0, 9.0])

In [11]:
id(v1) # Vector 객체가 새로 생성

2315758510024

In [12]:
v1_alias

Vector([1.0, 2.0, 3.0])

In [13]:
v1 *= 11 # 곱셈 할당자 실행
v1

Vector([55.0, 77.0, 99.0])

In [14]:
id(v1)

2315758550856

<hr>

- 인프레이스 연산자를 구현하지 않으면 복합할당 연산자는 단지 편의 구문으로서
    - a += b를 a = a + b와 동일하게 평가
- \__add\__() 매서드가 구현되어 있다면 아무런 코드를 추가하지 않고도 += 연산자 작동

- \__iadd\__() 등의 인플레이스 연산자 메서드 정의한 경우
    - 새로운 객체를 생성하지 않고 왼쪽에 나온 피연산자를 직접 변경

#### 13-16 AddableBingoCage 객체 사용법

In [18]:
vowels = "AEIOU" # 항목 다섯 개 (각기 모음에 해당)를 가진 globe 객체를 생성
globe = AddableBingoCage(vowels)
globe.inspect()

('A', 'E', 'I', 'O', 'U')

In [19]:
globe.pick() in vowels # 항목 하나를 꺼내서 모음 문자 인지 확인

True

In [20]:
len(globe.inspect()) # globe의 항목이 네 개로 줄었는지 확인

4

In [21]:
globe2 = AddableBingoCage('XYZ') # 항목을 3개 가진 두 번째 객체 생성
globe3 = globe + globe2 # 앞의 객체 두 개를 더해서 세 번째 객체를 생성, 이 객체는 일곱 개의 항목을 가짐

In [22]:
len(globe3.inspect())

7

In [23]:
void = globe + [10, 20] # AddableBingoCage를 list에 더하려고 시도하면 TypeError 발생
# __add__() 메서드가 NotImplemented를 반환한 후 파이썬 인터프리터가 실행

TypeError: unsupported operand type(s) for +: 'AddableBingoCage' and 'list'

<hr>

#### 예제 13-17 += 연산자를 사용해서 기존 AddableBingoCage 객체에 항목추가하기

In [24]:
globe_orig = globe # 별명을 생성해서 나중에 객체의 정체성 확인 가능하게 함
len(globe.inspect()) # globe는 4개의 항목을 가지고 있음

4

In [25]:
globe += globe2 # AddableBingoCage 객체는 동일한 클래스의 다른 객체에서 항목을 받을 수 있음
len(globe.inspect())

7

In [26]:
globe += ["M", "N"] # += 연산자 오른쪽 피연산자에는 어떠한 반복형이라도 올 수 있음
len(globe.inspect())

9

In [27]:
globe is globe_orig # 이 예제 내내 globe는 globe_orig 객체를 참조

True

In [28]:
globe += 1 # 비반복형을 AddableBingoCage에 추가하면 에러 메세지 발생

TypeError: right operand in += must be 'AddableBingoCage' or an iterable

<hr>

- 두 번째 피연산자의 측면에서 보면 += 연산자가 + 연산자보다 자유로움
    - + 연산자의 경우 서로 다른 자료형을 받으면 결과가 어떤 자료형이 되어야하는지 혼란스러움
    - += 연산자의 경우는 왼쪽 객체의 내용이 갱신되므로, 연산 결과 자료형이 명확
    

In [15]:
import abc

class Tombola(abc.ABC):  # <1>

    @abc.abstractmethod
    def load(self, iterable):  # <2>
        """Add items from an iterable."""

    @abc.abstractmethod
    def pick(self):  # <3>
        """Remove item at random, returning it.
        This method should raise `LookupError` when the instance is empty.
        """

    def loaded(self):  # <4>
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())  # <5>

    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:  # <6>
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)  # <7>
        return tuple(sorted(items))

In [16]:
import random

class BingoCage(Tombola):  
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom()  
        self._items = []
        self.load(items)  

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)  

    def pick(self):  
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):  
        self.pick()

#### 예제 13-18  bingoaddable.py : +와 += 을 지원하기 위해 BingoCage를 확장한 AddableBingoCage 클래스

In [17]:
class AddableBingoCage(BingoCage):   # AddableBingoCage 클래스는 BingoCage 클래스를 확장

    def __add__(self, other):
        if isinstance(other, Tombola):  # __add__() 매서드는 두번째 연산자가 Tombola 객체일 때만 작동 
            return AddableBingoCage(self.inspect() + other.inspect()) # other 객체에서 항목을 가져옴
        else:
            return NotImplemented

    def __iadd__(self, other):
        if isinstance(other, Tombola):
            other_iterable = other.inspect()  
        else:
            try:
                other_iterable = iter(other)  # Tombola 객체가 아닐 때는 other의 반복자를 가져옴
            except TypeError:  # 실패하면 메세지와 함꼐 예외 발생, 가능하면 해결 방법을 자세하게 써주는 것이 좋음
                self_cls = type(self).__name__
                msg = ('right operand in += must be {!r} or an iterable' )
                raise TypeError(msg.format(self_cls))
                
        self.load(other_iterable) # other_iterable을 self에 로딩 가능  
        return self # 할당 연산 특별 메소드는 반드시 self를 반환

**\__add\__()**
- AddableBingoCage()를 호출해서 생성된 새로운 객체 반환

<br>

**\__iadd\__()**
- 객체 자신을 변경한 후 self를 반환


- AddableBincoCage에는 \__radd\__() 가 구현되지 않음
- 정방향 메서드 \__add\__()는 오른쪽에도 동일한 자료형 객체가 와야 작동
    - AddableBincoCage 인 a와 AddableBincoCage 가 아닌 b 생성
    - a+b 계산
    - NotImplemented 반환
    - b 객체의 클래스가 이 연산 처리 가능
    
    <br>
    
    - b+a 계산
    - NotImplemented 반환
    - 파이썬이 TypeError를 발생시키고 포기하는 것이 나음
    - b 객체는 처리할수 없기 때문